In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")

database = PineconeVectorStore.from_documents([], embedding, index_name=index_name)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전 : {dictionary}

    질문 : {{question}}                           
""")

dictionary_chain = prompt | llm | StrOutputParser()

tax_chain = {"query": dictionary_chain} | qa_chain

In [69]:
ai_message = qa_chain({"query": query})

In [70]:
ai_message

{'query': '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.',
 'result': '연봉 5천만원의 경우, 소득세는 과세표준 구간 1,400만원 초과 5,000만원 이하에 해당합니다. 따라서 세액은 기본 84만원에다가, 1,400만원을 초과하는 금액인 3,600만원(5,000만원 - 1,400만원)의 15%를 더하게 됩니다. 계산하면 총 소득세는 84만원 + 540만원(3,600만원 x 15%) = 624만원입니다.'}

In [76]:
new_question = dictionary_chain.invoke({"question": query})

In [77]:
new_question

'질문 : 연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [78]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [79]:
ai_response = tax_chain.invoke({"question": query})

In [80]:
ai_response

{'query': '거주자 연봉 5천만원의 소득세는 얼마인가요?',
 'result': '거주자의 연봉 5,000만원에 대한 소득세는 624만원입니다. 이 금액은 종합소득 과세표준에서 5,000만원 이하의 구간에 해당하며, 해당 구간의 계산식은 "84만원 + (1,400만원을 초과하는 금액의 15퍼센트)"를 적용한 결과입니다.'}